# Introduction


Sigma seems like a really great project that is developing and sharing security content in a standardized format.

* https://github.com/Neo23x0/sigma

In this notebook I will try to test run all the sigma rules agains the botsv2 dataset. I didn't fully understand the sigmac commandline configuration options and therefore used the tool without extra configuration and instead modified the rules with python in this notebook.  This was just the fastest way to create some rules that i could add run more splunk queries on, and there are probably better ways to do it.


# Notebook setup

In [272]:
import os
import yaml
import time
import intake

import pandas as pd
import numpy as np

from pathlib import Path
from multiprocessing import Pool
from tqdm import tqdm_notebook
from intake_splunk.core import SplunkConnect

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 0)

s=SplunkConnect("https://localhost:8089")
s.auth("admin","superadmin")
SPL=s.read_pandas

# Sigma


In [8]:
%cd /home/ole/sigma/tools/

/home/ole/sigma/tools


In [327]:
#Function for building a splunk search based on a sigma rule. This is a bit of a hack.

def opensiemrule(sigmafile):
    with open(sigmafile, 'r') as stream:
        try:
            sigma=yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
            return
    
    sigma_spl=!./sigmac --target splunk -c splunk-windows-all $sigmafile
    
    
    #Build spl for opensiem rule
    spl=""
    spl+="index=botsv2\n"
    spl+=sigma_spl[0]
    
    if "title" in sigma:
        spl+='\n| eval s.title="{title}"'.format(**sigma)
    if "description" in sigma:
        spl+='\n| eval s.description="{description}"'.format(**sigma)
    if "status" in sigma:
        spl+='\n| eval s.status="{status}"'.format(**sigma)
    if "level" in sigma:
        spl+='\n| eval s.level="{level}"'.format(**sigma)
    if "tags" in sigma:
        if len(sigma["tags"])==1:
            spl+='\n| eval s.tags="{tags}"'.format(**sigma)
        else:
            tags=",".join('"' +tag+'"' for tag in sigma["tags"])
            spl+=f'\n| eval s.tags=mvappend({tags})'
    spl+="\n| eval opensiem_time=now() "
    spl+="\n| mkjson includehidden=true" 
    spl+="\n| collect index=opensiem sourcetype=opensiem addtime=false"
    return spl

In [325]:
#Example splunk search a sigma rule
print(opensiemrule("/home/ole/sigma/rules/windows/process_creation/win_susp_powershell_empire_lanuch.yml"))

index=botsv2
(CommandLine="* -NoP -sta -NonI -W Hidden -Enc *" OR CommandLine="* -noP -sta -w 1 -enc *")
| eval s.title="Empire PowerShell Launch Parameters"
| eval s.description="Detects suspicious powershell command line parameters used in Empire"
| eval s.status="experimental"
| eval s.level="critical"
| eval s.tags=mvappend("attack.execution","attack.t1086")
| eval opensiem_time=now() 
| mkjson includehidden=true
| collect index=opensiem sourcetype=opensiem addtime=false


In [319]:
%%time

#Load all rules from signa rules folder and create splunk searhes
rules={}
results={}

directory="/home/ole/sigma/rules/" 
i=0
for dirpath, dirnames, files in os.walk(directory):
    for name in files:
        if name.endswith(".yml"):
            filename=Path(dirpath)/name
            print (filename)
            rules[name]=opensiemrule(filename)
            #print(rules[name])                           

/home/ole/sigma/rules/proxy/proxy_ua_apt.yml
/home/ole/sigma/rules/proxy/proxy_downloadcradle_webdav.yml
/home/ole/sigma/rules/proxy/proxy_cobalt_onedrive.yml
/home/ole/sigma/rules/proxy/proxy_ua_hacktool.yml
/home/ole/sigma/rules/proxy/proxy_ua_malware.yml
/home/ole/sigma/rules/proxy/proxy_telegram_api.yml
/home/ole/sigma/rules/proxy/proxy_ua_suspicious.yml
/home/ole/sigma/rules/proxy/proxy_cobalt_ocsp.yml
/home/ole/sigma/rules/proxy/proxy_susp_flash_download_loc.yml
/home/ole/sigma/rules/proxy/proxy_empty_ua.yml
/home/ole/sigma/rules/proxy/proxy_apt40.yml
/home/ole/sigma/rules/proxy/proxy_chafer_malware.yml
/home/ole/sigma/rules/proxy/proxy_download_susp_tlds_whitelist.yml
/home/ole/sigma/rules/proxy/proxy_powershell_ua.yml
/home/ole/sigma/rules/proxy/proxy_ua_bitsadmin_susp_tld.yml
/home/ole/sigma/rules/proxy/proxy_cobalt_amazon.yml
/home/ole/sigma/rules/proxy/proxy_ua_frameworks.yml
/home/ole/sigma/rules/proxy/proxy_download_susp_dyndns.yml
/home/ole/sigma/rules/proxy/proxy_downloa

/home/ole/sigma/rules/windows/process_creation/win_exploit_cve_2017_8759.yml
/home/ole/sigma/rules/windows/process_creation/win_susp_execution_path_webserver.yml
/home/ole/sigma/rules/windows/process_creation/win_susp_sysvol_access.yml
/home/ole/sigma/rules/windows/process_creation/win_termserv_proc_spawn.yml
/home/ole/sigma/rules/windows/process_creation/win_susp_gup.yml
/home/ole/sigma/rules/windows/process_creation/win_mshta_spawn_shell.yml
/home/ole/sigma/rules/windows/process_creation/win_susp_script_execution.yml
/home/ole/sigma/rules/windows/process_creation/win_susp_rundll32_activity.yml
/home/ole/sigma/rules/windows/process_creation/win_exploit_cve_2015_1641.yml
/home/ole/sigma/rules/windows/process_creation/win_susp_powershell_enc_cmd.yml
/home/ole/sigma/rules/windows/process_creation/win_lethalhta.yml
/home/ole/sigma/rules/windows/process_creation/win_susp_procdump.yml
/home/ole/sigma/rules/windows/process_creation/win_susp_bcdedit.yml
/home/ole/sigma/rules/windows/process_c

/home/ole/sigma/rules/windows/sysmon/sysmon_quarkspw_filedump.yml
/home/ole/sigma/rules/windows/sysmon/sysmon_susp_prog_location_network_connection.yml
/home/ole/sigma/rules/windows/sysmon/sysmon_susp_reg_persist_explorer_run.yml
/home/ole/sigma/rules/windows/sysmon/sysmon_cmstp_execution.yml
expected a single document in the stream
  in "/home/ole/sigma/rules/windows/sysmon/sysmon_cmstp_execution.yml", line 2, column 1
but found another document
  in "/home/ole/sigma/rules/windows/sysmon/sysmon_cmstp_execution.yml", line 24, column 1
/home/ole/sigma/rules/windows/sysmon/sysmon_tsclient_filewrite_startup.yml
/home/ole/sigma/rules/windows/sysmon/sysmon_password_dumper_lsass.yml
/home/ole/sigma/rules/windows/sysmon/sysmon_cobaltstrike_process_injection.yml
/home/ole/sigma/rules/windows/sysmon/sysmon_win10_sched_task_0day.yml
/home/ole/sigma/rules/windows/sysmon/sysmon_ssp_added_lsa_config.yml
/home/ole/sigma/rules/windows/sysmon/sysmon_win_binary_susp_com.yml
/home/ole/sigma/rules/window

In [322]:
#Function for running splunk rule

def runspl(rule):
    name,spl=rule
    try:
        if not name in results:
            print ("Running:" + name)
            result=SPL(rules[name])
        else:
            result=None
            print ("Skipping:" + name)
    except pd.errors.EmptyDataError:
        result=pd.DataFrame() 
    except Exception as e:
        print("Error: " + str(e))
        print(name)
        print(spl)
        result=None

    return name,spl,result

In [323]:
#This will execute all rules on a splunk cluster. It Took about 18min to run all 307 rules with 12 parallell jobs.
#Lots of posibilities for optimzing with better sourcetyping and indexing

%%time
with Pool(12) as pool:
    for name,spl,result in tqdm_notebook(pool.imap_unordered(runspl,list(rules.items())),total=len(rules)):
        if result is not None:
            print ("Finished " + name)
            results[name]=result

Running:proxy_ua_apt.yml


Running:proxy_downloadcradle_webdav.yml
Running:proxy_cobalt_onedrive.yml
Running:proxy_telegram_api.yml
Running:proxy_ua_hacktool.yml
Running:proxy_ua_suspicious.yml
Running:proxy_ua_malware.yml
Running:proxy_susp_flash_download_loc.yml
Running:proxy_cobalt_ocsp.yml
Running:proxy_empty_ua.yml
Running:proxy_apt40.yml
Running:proxy_chafer_malware.yml
Running:proxy_download_susp_tlds_whitelist.yml
Finished proxy_cobalt_ocsp.yml
Running:proxy_powershell_ua.yml
Running:proxy_ua_bitsadmin_susp_tld.yml
Running:proxy_cobalt_amazon.yml
Finished proxy_cobalt_onedrive.yml
Finished proxy_downloadcradle_webdav.yml
Finished proxy_susp_flash_download_loc.yml
Running:proxy_ua_frameworks.yml
Running:proxy_download_susp_dyndns.yml
Finished proxy_telegram_api.yml
Finished proxy_ua_bitsadmin_susp_tld.yml
Running:proxy_download_susp_tlds_blacklist.yml
Running:av_webshell.yml
Running:av_relevant_files.yml
Finished proxy_apt40.yml
Finished proxy_cobalt_amazon.yml
Finished proxy_chafer_malware.yml
Running:av

Running:win_exploit_cve_2015_1641.yml
Finished win_susp_rundll32_activity.yml
Running:win_susp_powershell_enc_cmd.yml
Finished win_exploit_cve_2015_1641.yml
Running:win_lethalhta.yml
Finished win_susp_powershell_enc_cmd.yml
Running:win_susp_procdump.yml
Finished win_lethalhta.yml
Running:win_susp_bcdedit.yml
Finished win_susp_procdump.yml
Running:win_susp_outlook.yml
Finished win_susp_bcdedit.yml
Running:win_wmi_persistence_script_event_consumer.yml
Finished proxy_empty_ua.yml
Running:win_shell_spawn_susp_program.yml
Finished win_wmi_persistence_script_event_consumer.yml
Running:win_susp_mmc_source.yml
Finished win_shell_spawn_susp_program.yml
Running:win_susp_rasdial_activity.yml
Finished win_susp_mmc_source.yml
Running:win_lm_namedpipe.yml
Finished win_susp_rasdial_activity.yml
Running:win_susp_backup_delete.yml
Finished win_lm_namedpipe.yml
Running:win_susp_add_sid_history.yml
Finished win_susp_backup_delete.yml
Running:win_susp_security_eventlog_cleared.yml
Finished win_susp_add_si

Running:sysmon_malware_backconnect_ports.yml
Running:sysmon_logon_scripts_userinitmprlogonscript.yml
Finished sysmon_cobaltstrike_process_injection.yml
Finished sysmon_win10_sched_task_0day.yml
Running:sysmon_ghostpack_safetykatz.yml
Finished sysmon_ssp_added_lsa_config.yml
Running:sysmon_susp_file_characteristics.yml
Running:sysmon_rundll32_net_connections.yml
Finished sysmon_win_binary_susp_com.yml
Finished sysmon_powershell_exploit_scripts.yml
Running:sysmon_lsass_memdump.yml
Running:sysmon_susp_powershell_rundll32.yml
Finished sysmon_malware_backconnect_ports.yml
Finished sysmon_logon_scripts_userinitmprlogonscript.yml
Running:sysmon_susp_run_key_img_folder.yml
Running:sysmon_wmi_susp_scripting.yml
Finished sysmon_rundll32_net_connections.yml
Finished sysmon_lsass_memdump.yml
Running:sysmon_wmi_event_subscription.yml
Finished sysmon_susp_powershell_rundll32.yml
Running:powershell_shellcode_b64.yml
Finished win_pass_the_hash_2.yml
Running:powershell_exe_calling_ps.yml
Running:powers

/home/ole/anaconda3/lib/python3.7/multiprocessing/pool.py:121: DtypeWarning: Columns (132,275,328,337,463,465,534) have mixed types. Specify dtype option on import or set low_memory=False.
  result = (True, func(*args, **kwds))


Running:apt_stonedrill.yml
Running:apt_babyshark.yml
Finished lnx_shell_priv_esc_prep.yml
Finished apt_hurricane_panda.yml
Running:crime_fireball.yml
Finished apt_stonedrill.yml
Running:apt_carbonpaper_turla.yml
Running:apt_turla_commands.yml
Error: 'NoneType' object has no attribute 'strip'
apt_turla_commands.yml
None
Running:apt_turla_namedpipes.yml
Finished crime_fireball.yml
Finished apt_babyshark.yml
Running:apt_apt29_thinktanks.yml
Running:apt_tropictrooper.yml
Finished apt_carbonpaper_turla.yml
Finished apt_turla_namedpipes.yml
Running:apt_equationgroup_lnx.yml
Finished apt_apt29_thinktanks.yml
Running:web_apache_threading_error.yml
Running:web_multiple_suspicious_resp_codes_single_source.yml
Finished apt_equationgroup_lnx.yml
Finished lnx_shell_susp_rev_shells.yml
Running:web_apache_segfault.yml
Finished web_apache_threading_error.yml
Running:web_source_code_enumeration.yml
Finished apt_equationgroup_c2.yml
Running:web_cve_2018_2894_weblogic_exploit.yml
Finished web_apache_segf

In [ ]:
# Run without multiprocessing for easier debugging
#%%time
#for rule in tqdm_notebook((rules.items())):
#    name,spl,result=runspl(rule)
#    if result is not None:
#            print ("Finished " + name)
#            results[name]=result   

# Results

In [328]:
# Genereated 6830 alarms
SPL("index=opensiem | stats count")

,count
0,6830


In [332]:
#13 diffent signatures that triggered
SPL("index=opensiem | stats count by s.title,s.level,s.status | sort -count")

,s.title,s.level,s.status,count
0,PowerShell Network Connections,low,experimental,4572
1,Privilege Escalation Preparation,medium,experimental,1263
2,NotPetya Ransomware Activity,critical,experimental,484
3,Windows Processes Suspicious Parent Directory,low,experimental,299
4,Empire PowerShell Launch Parameters,critical,experimental,114
5,Unauthorized System Time Modification,high,experimental,10
6,Detects local user creation,low,experimental,4
7,Default PowerSploit Schtasks Persistence,high,experimental,3
8,Scheduled Task Creation,low,experimental,3
9,Windows Shell Spawning Suspicious Program,high,experimental,3


In [333]:
#14 hosts with alarms
SPL("index=opensiem | stats count by host | sort -count")

,host,count
0,soc1,6830
1,venus,4613
2,jabbah,1087
3,cassiopeia,177
4,wrk-btun,80
5,wrk-klagerf,78
6,wrk-aturing,54
7,wrk-abungst,43
8,wrk-ghoppy,42
9,wrk-bgist,35
